In [1]:
from src.acoustic_analyser import frame


test_frame = frame.from_file(member_file="test_member.json", constraint_file="test_constraint.csv")

free_end = test_frame.fixed_end(member_id=0)
free_end = test_frame.free_end(member_id=1)

matrix = test_frame.get_equation_matrix()

2022-10-16 19:44:12,510 src.modules.member DEBUG:Propagation Matrix Calculated for member 0
2022-10-16 19:44:12,514 src.modules.member DEBUG:Parameters set for member 0
2022-10-16 19:44:12,515 src.acoustic_analyser DEBUG:Member added with id 0
2022-10-16 19:44:12,517 src.modules.member DEBUG:Propagation Matrix Calculated for member 1
2022-10-16 19:44:12,519 src.modules.member DEBUG:Parameters set for member 1
2022-10-16 19:44:12,520 src.acoustic_analyser DEBUG:Member added with id 1
2022-10-16 19:44:12,522 src.acoustic_analyser DEBUG:Created all Members
2022-10-16 19:44:12,541 src.modules.rt_joint DEBUG:Equation File Loaded
2022-10-16 19:44:12,682 src.modules.rt_joint DEBUG:M0-M6 Substituted
2022-10-16 19:44:12,683 src.modules.rt_joint DEBUG:Matrix([[0, 0, -I/w**0.5 + 12*w**0.5], [12*w**0.5 - I, 12*w**0.5 + 1, 0], [1, -1, 0]])
2022-10-16 19:44:26,862 src.modules.rt_joint DEBUG:Reflection Transmission Calculated
2022-10-16 19:44:26,864 src.acoustic_analyser DEBUG:Two member joint added 

In [2]:
matrix.shape

(12, 12)